# SBTi-Finance Tool - FINZ Portfolio Coverage Calculator

This notebook calculates **FINZ climate alignment coverage** — the percentage of your investment portfolio that holds companies with validated 1.5°C Science Based Targets.

**FINZ Climate Alignment Assessment:**
- **In Transition**: Companies with validated 1.5°C SBTi targets
- **Assessed**: All other companies (including WB2°C, 2°C, or no target)

**Coverage is measured two ways (no weighting, simple per-entity basis):**
1. **By Investment Value ($)** — $ covered / total $ portfolio value
2. **By Company Count (#)** — # companies / total # companies

The granular **target classification** (1.5°C, WB2°C, 2°C, etc.) is preserved in the output for detailed analysis.

# Quick Start Guide

## What This Notebook Does
This notebook calculates **FINZ climate alignment coverage** - the percentage of your investment portfolio that holds companies with validated 1.5°C Science Based Targets.

---

## FINZ Climate Alignment Assessment

Per the **FINZ Standard Table 4.2**, this notebook outputs climate alignment categorization using **SBTi Target Status** as the "In Transition" methodology:

| Category | Definition |
|----------|------------|
| **In Transition** | Companies with validated 1.5°C SBTi targets |
| **Assessed** | All other companies (including WB2°C, 2°C, or no target) |

**Coverage Metrics (no weighting, simple per-entity basis):**
- **$ Coverage** = Investment value in category / Total portfolio value
- **# Coverage** = Number of companies in category / Total company count

The granular **target classification** (1.5°C, WB2°C, 2°C, etc.) is preserved in the output for detailed analysis.

> **Note:** This notebook uses SBTi 1.5°C Target Status as the sole "In Transition" methodology. 
> Full FINZ.Metric.2 calculation would also include other third-party methodologies (MSCI ITR, TPI, etc.), 
> taxonomy alignment ("Climate Solutions"), and verified GHG inventories ("Net-zero State").

---

## How to Use This Notebook

### Step 1: Run the Setup
Run the first few cells to install packages and load sample data.

### Step 2: Upload Your Data (optional)
To analyze your own portfolio:

| Environment | How to Upload |
|-------------|---------------|
| **Google Colab** | Click folder icon in left sidebar > Navigate to `data/` > Drag & drop your file |
| **Local/Jupyter** | Copy your file to the `examples/data/` folder in your notebook directory |

Then update the file path in the "Load Your Portfolio" section.

### Step 3: Run All Cells
- **Google Colab:** Click **Runtime > Run all** from the menu
- **Local/Jupyter:** Click **Cell > Run All** or use keyboard shortcut (Shift+Enter for each cell)

### Step 4: Download Results
Your results will be saved as an Excel file in the `data/` folder with:
- **Portfolio sheet:** Per-company FINZ alignment categorization with target classification
- **Summary sheet:** Coverage by $ value and # companies

---

## Required Data Format
Your portfolio file needs these columns:

| Column | Required | Example | Description |
|--------|----------|---------|-------------|
| `company_name` | Yes | "Apple Inc." | Company name for matching |
| `company_id` | Yes | "AAPL" | Your internal identifier |
| `investment_value` | Yes | 1000000 | Investment amount (any currency) |
| `isin` | Recommended | "US0378331005" | International Securities ID (improves match accuracy) |
| `lei` | Recommended | "HWUPKR0MPOU8FGXBT394" | Legal Entity Identifier (most reliable match) |

**Tip:** Including ISIN and/or LEI significantly improves matching accuracy compared to company name alone.

---

In [ ]:
# =============================================================================
# SETUP - Run this cell first
# =============================================================================

import sys

# Detect environment and install/import accordingly
if 'google.colab' in sys.modules:
    print("=" * 50)
    print("  GOOGLE COLAB ENVIRONMENT DETECTED")
    print("=" * 50)
    print("\nInstalling SBTi Finance Tool...")
    !pip install -q sbti-finance-tool
    print("✓ Installation complete")
else:
    print("=" * 50)
    print("  LOCAL/JUPYTER ENVIRONMENT DETECTED")
    print("=" * 50)
    print("\nUsing locally installed packages")
    print("Tip: Run 'pip install sbti-finance-tool' if not installed")

# Import required libraries
import pandas as pd
import openpyxl
from datetime import datetime
import os
import re
import warnings

# Suppress common warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=FutureWarning)

print("\n✓ All packages loaded successfully")
print(f"\nEnvironment: {'Google Colab' if 'google.colab' in sys.modules else 'Local/Jupyter'}")
print("Ready to proceed!")

## Download Sample Data
This cell downloads example data so you can test the notebook. Skip this if using your own data.

In [ ]:
# Download sample portfolio data
import urllib.request

if not os.path.isdir("data"):
    os.mkdir("data")
    
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve(
        "https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", 
        "data/example_portfolio.csv"
    )
    print("Sample data downloaded to data/example_portfolio.csv")
else:
    print("Sample data already exists")

## USER INPUT: Load Your Portfolio

### Option 1: Use Sample Data (Default)
Just run the next cell as-is to use the included example portfolio.

### Option 2: Use Your Own Data

**Google Colab:**
1. Click the folder icon (📁) in the left sidebar
2. Navigate to `data/` folder  
3. Drag and drop your CSV or Excel file
4. Update the filename in the cell below

**Local/Jupyter:**
1. Copy your file to the `examples/data/` folder
2. Update the filename in the cell below

**Supported formats:** `.csv`, `.xlsx`

In [ ]:
# =============================================================================
# LOAD PORTFOLIO DATA
# =============================================================================

# --- SAMPLE DATA (default) ---
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

# --- YOUR OWN DATA ---
# Uncomment ONE of the lines below and update the filename:
#
# For CSV files:
# df_portfolio = pd.read_csv("data/YOUR_FILE.csv", encoding="utf-8")
#
# For Excel files:
# df_portfolio = pd.read_excel("data/YOUR_FILE.xlsx", engine="openpyxl")

print(f"Loaded portfolio file successfully")

In [ ]:
# Standardize column names (handles different naming formats automatically)
def convert_to_snake_case(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    s3 = s2.lower()
    s4 = re.sub(r'[^a-z0-9_]', '_', s3)
    s5 = re.sub(r'_+', '_', s4)
    return s5.strip('_')

df_portfolio.columns = [convert_to_snake_case(col) for col in df_portfolio.columns]
print(f"Loaded {len(df_portfolio)} companies from portfolio")

In [ ]:
# Auto-detect column formats (ISIN, LEI identifiers)
portfolio_isin_col = next((c for c in ['company_isin', 'isin'] if c in df_portfolio.columns), None)
portfolio_lei_col = next((c for c in ['company_lei', 'lei'] if c in df_portfolio.columns), None)

# Show what identifiers were found
id_status = []
if portfolio_isin_col:
    id_status.append("ISIN")
if portfolio_lei_col:
    id_status.append("LEI")
if id_status:
    print(f"Company identifiers found: {', '.join(id_status)}")
else:
    print("Warning: No ISIN or LEI columns found - will match by company name only")

In [ ]:
# Validate portfolio data
required_cols = ['company_name', 'company_id', 'investment_value']
missing_cols = [col for col in required_cols if col not in df_portfolio.columns]

if missing_cols:
    print(f"ERROR: Missing required columns: {missing_cols}")
    print(f"Your file needs: company_name, company_id, investment_value")
    raise ValueError(f"Missing columns: {missing_cols}")
else:
    total_value = df_portfolio['investment_value'].sum()
    print(f"Portfolio validated: {len(df_portfolio)} companies, ${total_value:,.0f} total value")

## USER INPUT: Select Analysis Date

Choose the date for your analysis. This determines which SBTi targets are included (only targets published by this date).

**Default:** December 31, 2025

In [ ]:
year = 2025 #enter the year for which you want to calculate the portfolio coverage
month = 12 #enter the month for which you want to calculate the portfolio coverage
day = 31 #enter the day for which you want to calculate the portfolio coverage

In [ ]:
user_date = datetime(year, month, day)

## Loading SBTi Database
The tool automatically downloads the latest list of companies with validated Science Based Targets.

In [ ]:
# Load SBTi Companies Taking Action database
from SBTi.data.sbti import SBTi

print("Loading SBTi database (this may take a moment)...")
sbti_provider = SBTi()
cta_file = sbti_provider.targets.copy()

# Count unique companies with targets
companies_with_targets_count = len(cta_file[cta_file[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET][sbti_provider.c.COL_COMPANY_NAME].unique())
print(f"Loaded SBTi database: {companies_with_targets_count:,} companies with validated targets")

## Processing Data
Filtering SBTi data to match your analysis date...

In [ ]:
# Filter for companies with validated targets
targets = cta_file.copy()
companies_with_targets = targets[targets[sbti_provider.c.COL_ACTION] == sbti_provider.c.VALUE_ACTION_TARGET]

print(f"Processing {len(companies_with_targets[sbti_provider.c.COL_COMPANY_NAME].unique()):,} companies with SBT targets")

In [ ]:
# Handle date column format variations (contingency for different CTA formats)
date_col = sbti_provider.c.COL_DATE_PUBLISHED
potential_date_cols = ['date_updated', 'Date Updated', 'date_published', 'Date Published']

# Check if expected date column exists, otherwise search for alternatives
if date_col not in companies_with_targets.columns:
    found_date_col = next((col for col in potential_date_cols if col in companies_with_targets.columns), None)
    if found_date_col:
        companies_with_targets = companies_with_targets.rename(columns={found_date_col: date_col})
        print(f"Date column mapped: '{found_date_col}' -> '{date_col}'")
    else:
        print(f"WARNING: No date column found. Available columns: {list(companies_with_targets.columns)}")
        print("Date filtering will be skipped.")
else:
    print(f"Date column: '{date_col}'")

In [ ]:
# Apply date filter
df_targets = companies_with_targets.copy()
df_targets[sbti_provider.c.COL_DATE_PUBLISHED] = pd.to_datetime(df_targets[sbti_provider.c.COL_DATE_PUBLISHED])

# Filter by user-specified date
date_filtered_df = df_targets.loc[df_targets[sbti_provider.c.COL_DATE_PUBLISHED] <= user_date]

# Name normalization function (handles whitespace variations)
def normalize_name(name):
    """Normalize company name for matching: lowercase, collapse whitespace."""
    if pd.isna(name):
        return None
    return ' '.join(str(name).lower().split())

# Create normalized company name set from ALL date-filtered companies
date_filtered_df['company_name_normalized'] = date_filtered_df[sbti_provider.c.COL_COMPANY_NAME].apply(normalize_name)
company_name_set = set(date_filtered_df['company_name_normalized'].dropna())

# Create ISIN/LEI sets only from companies that have these identifiers
isin_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_ISIN].dropna())
lei_set = set(date_filtered_df[sbti_provider.c.COL_COMPANY_LEI].dropna())

# Keep filtered_df for 1.5C analysis
filtered_df = date_filtered_df.copy()

print(f"Filtered to targets published by {user_date.strftime('%B %d, %Y')}")
print(f"  - Companies for name matching: {len(company_name_set):,}")
print(f"  - Unique ISINs: {len(isin_set):,}")
print(f"  - Unique LEIs: {len(lei_set):,}")

## Matching Your Portfolio
Now matching your portfolio companies against the SBTi database...

In [ ]:
# =============================================================================
# COMPANY MATCHING UTILITIES
# =============================================================================

def match_company(row, target_isin_set, target_lei_set, target_name_set):
    """
    Match a portfolio company against a set of target companies.
    
    Returns the match method ('LEI', 'ISIN', 'Name') or None if no match.
    
    Priority order (highest reliability first):
    1. LEI match (Legal Entity Identifier - most reliable)
    2. ISIN match (International Securities ID - very reliable)
    3. Company name match (normalized, case-insensitive - fallback)
    """
    # Check LEI first (most reliable identifier)
    if portfolio_lei_col and pd.notna(row.get(portfolio_lei_col)):
        if row.get(portfolio_lei_col) in target_lei_set:
            return 'LEI'

    # Check ISIN second (very reliable)
    if portfolio_isin_col and pd.notna(row.get(portfolio_isin_col)):
        if row.get(portfolio_isin_col) in target_isin_set:
            return 'ISIN'

    # Check company name last (normalized match)
    if pd.notna(row.get('company_name')):
        # Use same normalization as CTA names
        normalized_name = normalize_name(row.get('company_name'))
        if normalized_name and normalized_name in target_name_set:
            return 'Name'

    return None


def get_target_classification(row, target_df, classification_col):
    """
    Look up the target classification for a matched portfolio company.
    
    Uses waterfall matching: LEI > ISIN > Company Name (exact match after normalization)
    
    Returns the target classification (e.g., '1.5°C', 'Well-below 2°C', '2°C') or None.
    """
    # Check LEI first
    if portfolio_lei_col and pd.notna(row.get(portfolio_lei_col)):
        matches = target_df[target_df[sbti_provider.c.COL_COMPANY_LEI] == row.get(portfolio_lei_col)]
        if len(matches) > 0:
            classification = matches.iloc[0][classification_col]
            if pd.notna(classification):
                return str(classification)
    
    # Check ISIN second
    if portfolio_isin_col and pd.notna(row.get(portfolio_isin_col)):
        matches = target_df[target_df[sbti_provider.c.COL_COMPANY_ISIN] == row.get(portfolio_isin_col)]
        if len(matches) > 0:
            classification = matches.iloc[0][classification_col]
            if pd.notna(classification):
                return str(classification)
    
    # Check company name last (normalized exact match)
    if pd.notna(row.get('company_name')):
        normalized_name = normalize_name(row.get('company_name'))
        if normalized_name:
            matches = target_df[target_df['company_name_normalized'] == normalized_name]
            if len(matches) > 0:
                classification = matches.iloc[0][classification_col]
                if pd.notna(classification):
                    return str(classification)
    
    return None


# =============================================================================
# VALIDATE PORTFOLIO AGAINST ALL SBT TARGETS (DATE-FILTERED) - FINT
# =============================================================================

# Apply matching to determine if company has validated SBT target
df_portfolio['match_method_all'] = df_portfolio.apply(
    lambda row: match_company(row, isin_set, lei_set, company_name_set), 
    axis=1
)
df_portfolio['validated'] = df_portfolio['match_method_all'].notna()

# Diagnostic: Show match method breakdown
match_counts = df_portfolio['match_method_all'].value_counts()
total_matched = df_portfolio['validated'].sum()
total_companies = len(df_portfolio)

print(f"SBT Matching (FINT): {total_matched} / {total_companies} companies matched")
print(f"  Match breakdown:")
for method in ['LEI', 'ISIN', 'Name']:
    count = match_counts.get(method, 0)
    if count > 0:
        print(f"    - {method}: {count}")

## USER INPUT: 1.5°C Target Settings (Optional)

**What is a 1.5°C target?** Companies with the most ambitious climate commitments, aligned with limiting global warming to 1.5°C. These are the only targets that count toward **FINZ (FI Net-Zero Standard)** coverage.

**Default settings work for most users.** Only change if you need specific filtering.

In [ ]:
# 1.5°C Target Configuration (most users can leave defaults)
# --------------------------------------------------------
# INCLUDE_PURE_15C = True   -> Count companies with pure 1.5°C targets
# INCLUDE_MIXED_15C_2C = False -> Exclude mixed targets like "1.5°C/2°C"

INCLUDE_PURE_15C = True        # Recommended: True
INCLUDE_MIXED_15C_2C = False   # Recommended: False (more conservative)

print(f"1.5°C settings: Pure={INCLUDE_PURE_15C}, Mixed={INCLUDE_MIXED_15C_2C}")

In [ ]:
# Identify 1.5°C aligned companies in SBTi database (for FINZ coverage)
df_analysis = filtered_df.copy()

# Find Target Classification column (handle potential naming variations)
TARGET_CLASSIFICATION_COL = None
potential_tc_cols = ['Target Classification', 'target_classification', 'Near Term Classification', 
                     'near_term_target_classification', 'target_classification_short']
for col in potential_tc_cols:
    if col in df_analysis.columns:
        TARGET_CLASSIFICATION_COL = col
        break

if TARGET_CLASSIFICATION_COL is None:
    print("ERROR: No Target Classification column found!")
    print(f"Available columns: {list(df_analysis.columns)}")
    raise ValueError("Missing Target Classification column")

# Show available classification values for transparency
unique_classifications = df_analysis[TARGET_CLASSIFICATION_COL].dropna().unique()
print(f"Target Classification column: '{TARGET_CLASSIFICATION_COL}'")
print(f"Classifications found: {sorted([str(x) for x in unique_classifications])}")

# Define known 1.5°C classification patterns (per FINZ Implementation List Table 1)
PURE_15C_VALUES = ['1.5°C', '1.5°C/1.5°C']  # Pure 1.5°C (both near-term and long-term)
MIXED_15C_PATTERNS = ['1.5°C/Well-below 2°C', '1.5°C/2°C', '1.5/']  # Mixed with other temps

# Filter for 1.5°C targets based on settings
df_1_5c = pd.DataFrame()

if INCLUDE_PURE_15C:
    # Match known pure 1.5°C values
    pure_mask = df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES)
    pure_15c = df_analysis[pure_mask]
    df_1_5c = pd.concat([df_1_5c, pure_15c])
    print(f"  Pure 1.5°C matches: {len(pure_15c)} rows")
    
    # Check for unexpected pure 1.5°C patterns (safety check)
    unexpected_pure = df_analysis[
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.startswith('1.5°C')) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES)) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('/', na=False))
    ]
    if len(unexpected_pure) > 0:
        print(f"  WARNING: Found {len(unexpected_pure)} rows with unexpected 1.5°C pattern")
        print(f"           Values: {unexpected_pure[TARGET_CLASSIFICATION_COL].unique()}")
    
if INCLUDE_MIXED_15C_2C:
    # Match mixed classifications containing 1.5°C with other temps
    mixed_mask = (
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('1.5', na=False)) &
        (df_analysis[TARGET_CLASSIFICATION_COL].astype(str).str.contains('/', na=False)) &
        (~df_analysis[TARGET_CLASSIFICATION_COL].isin(PURE_15C_VALUES))
    )
    mixed = df_analysis[mixed_mask]
    df_1_5c = pd.concat([df_1_5c, mixed])
    print(f"  Mixed 1.5°C matches: {len(mixed)} rows")

# Create lookup sets for 1.5°C matching
if len(df_1_5c) > 0:
    # Deduplicate by company name to count unique companies
    df_1_5c_unique = df_1_5c.drop_duplicates(subset=[sbti_provider.c.COL_COMPANY_NAME])
    
    # Create ISIN/LEI sets from companies that have these identifiers
    isin_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_ISIN].dropna())
    lei_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_LEI].dropna())
    
    # Create normalized name set from ALL 1.5°C companies
    company_name_set_1_5c = set(df_1_5c[sbti_provider.c.COL_COMPANY_NAME].apply(normalize_name).dropna())
    
    print(f"Found {len(df_1_5c_unique):,} unique companies with 1.5°C targets (FINZ eligible)")
    print(f"  - For name matching: {len(company_name_set_1_5c):,}")
    print(f"  - With ISIN: {len(isin_set_1_5c):,}")
    print(f"  - With LEI: {len(lei_set_1_5c):,}")
else:
    isin_set_1_5c, lei_set_1_5c, company_name_set_1_5c = set(), set(), set()
    print("Warning: No 1.5°C companies found with current settings")

In [ ]:
# =============================================================================
# MATCH PORTFOLIO COMPANIES TO 1.5°C ALIGNED COMPANIES (FINZ)
# =============================================================================

# Apply matching using the reusable match_company function
df_portfolio['match_method_1_5c'] = df_portfolio.apply(
    lambda row: match_company(row, isin_set_1_5c, lei_set_1_5c, company_name_set_1_5c), 
    axis=1
)
df_portfolio['is_1_5c'] = df_portfolio['match_method_1_5c'].notna()

# =============================================================================
# LOOK UP TARGET CLASSIFICATION FOR MATCHED COMPANIES
# =============================================================================

# Look up target classification for each portfolio company that matched
df_portfolio['target_classification'] = df_portfolio.apply(
    lambda row: get_target_classification(row, date_filtered_df, TARGET_CLASSIFICATION_COL) if row['validated'] else None,
    axis=1
)

# Diagnostic: Show 1.5°C match method breakdown
match_counts_15c = df_portfolio['match_method_1_5c'].value_counts()
total_matched_15c = df_portfolio['is_1_5c'].sum()
total_portfolio_companies = len(df_portfolio)

print(f"1.5°C Matching (FINZ): {total_matched_15c} / {total_portfolio_companies} companies")
if total_matched_15c > 0:
    print(f"  Match breakdown:")
    for method in ['LEI', 'ISIN', 'Name']:
        count = match_counts_15c.get(method, 0)
        if count > 0:
            print(f"    - {method}: {count}")

# Show target classification breakdown for matched companies
print(f"\nTarget Classifications found:")
classification_counts = df_portfolio['target_classification'].value_counts()
for classification, count in classification_counts.items():
    print(f"    - {classification}: {count}")

---
# Your Results: FINZ Climate Alignment Assessment

The following sections show your portfolio's **FINZ climate alignment** based on SBTi 1.5°C Target Status.

**FINZ Alignment Categories (per Table 4.2):**
- **In Transition** — Companies with validated 1.5°C SBTi targets
- **Assessed** — All other companies (including WB2°C, 2°C, or no validated target)

**Coverage is calculated on a simple per-entity basis (no weighting):**
- **$ Coverage** = Sum of investment values / Total portfolio value
- **# Coverage** = Count of companies / Total company count

> **Methodology:** This assessment uses **SBTi 1.5°C Target Status** as the "In Transition" methodology per FINZ Implementation List Table 1.

In [ ]:
# =============================================================================
# CALCULATE FINZ COVERAGE METRICS (No Weighting - Simple Per-Entity Basis)
# =============================================================================

total_investment_value = df_portfolio['investment_value'].sum()
total_companies = len(df_portfolio)
distinct_company_count = df_portfolio['company_name'].nunique()

# In Transition: 1.5°C targets only
in_transition_value = df_portfolio.loc[df_portfolio['is_1_5c'] == True, 'investment_value'].sum()
in_transition_count = df_portfolio['is_1_5c'].sum()

# Assessed: Everything else (including WB2°C, 2°C, and no target)
assessed_value = total_investment_value - in_transition_value
assessed_count = total_companies - in_transition_count

# Calculate percentages (simple $ and # coverage, no weighting)
in_transition_value_pct = (in_transition_value / total_investment_value * 100) if total_investment_value > 0 else 0
in_transition_count_pct = (in_transition_count / total_companies * 100) if total_companies > 0 else 0

assessed_value_pct = (assessed_value / total_investment_value * 100) if total_investment_value > 0 else 0
assessed_count_pct = (assessed_count / total_companies * 100) if total_companies > 0 else 0

# Display results
print("=" * 75)
print("           FINZ CLIMATE ALIGNMENT ASSESSMENT SUMMARY")
print("=" * 75)
print(f"\nTotal Portfolio Value: ${total_investment_value:,.0f}")
print(f"Total Companies: {total_companies}")

print(f"\n" + "=" * 75)
print("  COVERAGE BY CATEGORY (No Weighting - Simple Per-Entity Basis)")
print("=" * 75)

print(f"\n  IN TRANSITION (1.5°C validated SBTi targets)")
print(f"  " + "-" * 60)
print(f"    $ Coverage:  ${in_transition_value:,.0f} / ${total_investment_value:,.0f} = {in_transition_value_pct:.1f}%")
print(f"    # Coverage:  {in_transition_count} / {total_companies} = {in_transition_count_pct:.1f}%")

print(f"\n  ASSESSED (All other companies)")
print(f"  " + "-" * 60)
print(f"    $ Coverage:  ${assessed_value:,.0f} / ${total_investment_value:,.0f} = {assessed_value_pct:.1f}%")
print(f"    # Coverage:  {assessed_count} / {total_companies} = {assessed_count_pct:.1f}%")

print(f"\n" + "=" * 75)
print("  TARGET CLASSIFICATION BREAKDOWN (Granular Detail)")
print("=" * 75)
classification_breakdown = df_portfolio.groupby('target_classification').agg({
    'investment_value': 'sum',
    'company_name': 'count'
}).rename(columns={'company_name': 'count'})
classification_breakdown['value_pct'] = classification_breakdown['investment_value'] / total_investment_value * 100
classification_breakdown['count_pct'] = classification_breakdown['count'] / total_companies * 100

# Add "No Target" row for companies without classification
no_target_value = df_portfolio.loc[df_portfolio['target_classification'].isna(), 'investment_value'].sum()
no_target_count = df_portfolio['target_classification'].isna().sum()
if no_target_count > 0:
    print(f"\n  {'Classification':<25} {'$ Value':>15} {'$ %':>8} {'# Co.':>8} {'# %':>8}")
    print(f"  " + "-" * 65)
    for classification, row in classification_breakdown.iterrows():
        if pd.notna(classification):
            print(f"  {str(classification):<25} ${row['investment_value']:>13,.0f} {row['value_pct']:>7.1f}% {int(row['count']):>8} {row['count_pct']:>7.1f}%")
    print(f"  {'(No Target)':<25} ${no_target_value:>13,.0f} {no_target_value/total_investment_value*100:>7.1f}% {no_target_count:>8} {no_target_count/total_companies*100:>7.1f}%")
else:
    print(f"\n  {'Classification':<25} {'$ Value':>15} {'$ %':>8} {'# Co.':>8} {'# %':>8}")
    print(f"  " + "-" * 65)
    for classification, row in classification_breakdown.iterrows():
        print(f"  {str(classification):<25} ${row['investment_value']:>13,.0f} {row['value_pct']:>7.1f}% {int(row['count']):>8} {row['count_pct']:>7.1f}%")

print(f"\n" + "=" * 75)

In [ ]:
# =============================================================================
# VISUAL SUMMARY - FINZ Climate Alignment Assessment (2 Categories)
# =============================================================================

import matplotlib.pyplot as plt

# Import FINZ alignment categories from the library
from SBTi.interfaces import FINZAlignmentCategory

# Classify each company per FINZ alignment categories (Table 4.2)
# FINZ only has 2 categories: In Transition (1.5°C) and Assessed (everything else)
def get_finz_alignment(row):
    """
    Classify company climate alignment per FINZ Standard Table 4.2.
    
    Uses SBTi 1.5°C Target Status as the "In Transition" methodology
    per FINZ Implementation List Table 1.
    
    - In Transition: 1.5°C validated SBTi targets
    - Assessed: All other companies (WB2°C, 2°C, no target)
    """
    if row['is_1_5c']:
        return FINZAlignmentCategory.IN_TRANSITION.value
    else:
        return FINZAlignmentCategory.ASSESSED.value

df_portfolio['finz_category'] = df_portfolio.apply(get_finz_alignment, axis=1)

# Calculate values for chart (2 categories only)
alignment_values = df_portfolio.groupby('finz_category')['investment_value'].sum()

# Ensure order: In Transition, Assessed (2 categories)
categories = [
    FINZAlignmentCategory.IN_TRANSITION.value, 
    FINZAlignmentCategory.ASSESSED.value
]
values = [alignment_values.get(cat, 0) for cat in categories]
colors = ['#2ecc71', '#95a5a6']  # Green, Gray

# Create charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('FINZ Climate Alignment Assessment', fontsize=16, fontweight='bold', y=1.02)

# Pie chart by value
wedges, texts, autotexts = ax1.pie(
    values, 
    labels=categories, 
    autopct=lambda pct: f'{pct:.1f}%' if pct > 0 else '',
    colors=colors,
    startangle=90
)
ax1.set_title('Portfolio by Investment Value ($)', fontsize=14, fontweight='bold')

# Bar chart by company count
counts = df_portfolio.groupby('finz_category')['company_name'].count()
count_values = [counts.get(cat, 0) for cat in categories]

bars = ax2.bar(categories, count_values, color=colors)
ax2.set_ylabel('Number of Companies')
ax2.set_title('Portfolio by Company Count (#)', fontsize=14, fontweight='bold')

# Add value labels on bars
for bar, count in zip(bars, count_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Print summary table with both $ and # coverage
print("\n" + "=" * 80)
print("              FINZ CLIMATE ALIGNMENT ASSESSMENT")
print("=" * 80)
print(f"\nMethodology: SBTi 1.5°C Target Status (per FINZ Implementation List Table 1)")
print(f"Coverage Basis: Simple per-entity (no weighting)")
print(f"\n{'Category':<18} {'$ Value':>15} {'$ %':>10} {'# Companies':>15} {'# %':>10}")
print("-" * 80)
for cat, val, cnt in zip(categories, values, count_values):
    val_pct = (val / total_investment_value * 100) if total_investment_value > 0 else 0
    cnt_pct = (cnt / total_companies * 100) if total_companies > 0 else 0
    print(f"{cat:<18} ${val:>13,.0f} {val_pct:>9.1f}% {cnt:>15} {cnt_pct:>9.1f}%")
print("-" * 80)
print(f"{'TOTAL':<18} ${total_investment_value:>13,.0f} {'100.0':>9}% {total_companies:>15} {'100.0':>9}%")

print("\n" + "-" * 80)
print("FINZ COVERAGE (In Transition only):")
print(f"  $ Coverage: {in_transition_value_pct:.1f}% (${in_transition_value:,.0f} / ${total_investment_value:,.0f})")
print(f"  # Coverage: {in_transition_count_pct:.1f}% ({in_transition_count} / {total_companies} companies)")
print("\n  Note: Only 'In Transition' (1.5°C targets) counts toward FINZ alignment.")
print("        The granular target classification is preserved in the output file.")

---
# Download Your Results

Run the cell below to save your results as an Excel file.

## Output File
**Filename:** `portfolio_climate_alignment.xlsx`  
**Location:** `data/` folder

## How to Access Your File

| Environment | Instructions |
|-------------|--------------|
| **Google Colab** | 1. Click folder icon in left sidebar<br>2. Navigate to `data/` folder<br>3. Right-click `portfolio_climate_alignment.xlsx`<br>4. Select **Download** |
| **Local/Jupyter** | File is saved to the `examples/data/` folder in your notebook directory. The full path will be shown in the output below. |

## What's Included

| Sheet | Contents |
|-------|----------|
| **Portfolio** | Per-company data with FINZ alignment category (`In Transition` or `Assessed`) and granular target classification |
| **Summary** | Coverage metrics: $ value and # companies for each category |

In [ ]:
# =============================================================================
# SAVE RESULTS TO EXCEL - FINZ Climate Alignment Assessment (2 Categories)
# =============================================================================

output_filename = 'portfolio_climate_alignment'
output_dir = 'data'

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Prepare clean export data with FINZ alignment categories and target classification
export_columns = ['company_id', 'company_name', 'investment_value', 'finz_category', 'target_classification', 'match_method_all']
if portfolio_isin_col:
    export_columns.insert(3, portfolio_isin_col)
if portfolio_lei_col:
    export_columns.insert(3, portfolio_lei_col)

export_columns = [c for c in export_columns if c in df_portfolio.columns]
df_export = df_portfolio[export_columns].copy()

# Rename columns for clearer output
column_renames = {
    'match_method_all': 'match_method',
    'finz_category': 'finz_alignment_category'
}
df_export = df_export.rename(columns={k: v for k, v in column_renames.items() if k in df_export.columns})

# Create summary data with FINZ alignment assessment output (2 categories, $ and # coverage)
summary_data = {
    'Metric': [
        'FINZ CLIMATE ALIGNMENT ASSESSMENT', '', '',
        'Analysis Date', 'Coverage Date', 'Methodology', 'Coverage Basis', '',
        'PORTFOLIO TOTALS', '  Total Portfolio Value', '  Total Companies', '',
        '--- COVERAGE BY CATEGORY ---', '', '',
        f'{FINZAlignmentCategory.IN_TRANSITION.value}', '  Definition', '  $ Value', '  $ Coverage %', '  # Companies', '  # Coverage %', '',
        f'{FINZAlignmentCategory.ASSESSED.value}', '  Definition', '  $ Value', '  $ Coverage %', '  # Companies', '  # Coverage %', '',
        '--- FINZ COVERAGE SUMMARY ---', '  $ Coverage (In Transition)', '  # Coverage (In Transition)'
    ],
    'Value': [
        'SBTi 1.5°C Target Status Methodology', '', '',
        datetime.now().strftime('%Y-%m-%d'), user_date.strftime('%Y-%m-%d'), 'SBTi Target Status (FINZ Implementation List Table 1)', 'Simple per-entity (no weighting)', '',
        '', f"${total_investment_value:,.0f}", str(total_companies), '',
        '', '', '',
        '', '1.5°C validated SBTi targets', f"${in_transition_value:,.0f}", f"{in_transition_value_pct:.1f}%", str(in_transition_count), f"{in_transition_count_pct:.1f}%", '',
        '', 'All other companies (WB2°C, 2°C, no target)', f"${assessed_value:,.0f}", f"{assessed_value_pct:.1f}%", str(assessed_count), f"{assessed_count_pct:.1f}%", '',
        '', f"{in_transition_value_pct:.1f}% (${in_transition_value:,.0f} / ${total_investment_value:,.0f})", f"{in_transition_count_pct:.1f}% ({in_transition_count} / {total_companies} companies)"
    ]
}

# Save to Excel
excel_path = f"{output_dir}/{output_filename}.xlsx"

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    df_export.to_excel(writer, sheet_name='Portfolio', index=False)
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

# Get absolute path for local users
abs_path = os.path.abspath(excel_path)

print("=" * 65)
print("     FINZ CLIMATE ALIGNMENT ASSESSMENT - SAVED")
print("=" * 65)
print(f"\nFile saved to: {excel_path}")
print(f"\nContents:")
print(f"  - Portfolio sheet: Per-company FINZ alignment categories")
print(f"                     (In Transition or Assessed) + target classification")
print(f"  - Summary sheet: $ and # coverage metrics")

if 'google.colab' in sys.modules:
    print(f"\n[Google Colab] To download your file:")
    print(f"   1. Click the folder icon in the left sidebar")
    print(f"   2. Navigate to 'data' folder")
    print(f"   3. Right-click '{output_filename}.xlsx' > Download")
else:
    print(f"\n[Local] Full path: {abs_path}")
    print(f"   Open this location in your file explorer to access the file.")

print("\n" + "=" * 65)

# Show preview with FINZ alignment category and target classification
print(f"\nPreview of exported data:")
print(df_export.head(10).to_string(index=False))